In [1]:
from yt_dlp import YoutubeDL

In [2]:
list = [
    "https://www.youtube.com/watch?v=jLsii8s6acM&t=8s"
]

output_dir = "../data/videos"

In [7]:
def download_yt_video(video_url):
    yt_opts = {
        'format': 'best[ext=mp4]',  # Specify the format you want
        'outtmpl': '../data/videos/%(title)s.%(ext)s'
    }

    with YoutubeDL(yt_opts) as ytdl:
        ytdl.download(video_url)
    
    print(f'downloaded ''{title}''')

In [8]:
download_yt_video(list[0])

[youtube] Extracting URL: https://www.youtube.com/watch?v=jLsii8s6acM&t=8s
[youtube] jLsii8s6acM: Downloading webpage
[youtube] jLsii8s6acM: Downloading tv player API JSON
[youtube] jLsii8s6acM: Downloading ios player API JSON
[youtube] jLsii8s6acM: Downloading m3u8 information
[info] jLsii8s6acM: Downloading 1 format(s): 18
[download] How to Flag - A Climbing Technique for Achieving Balance [jLsii8s6acM].mp4 has already been downloaded
[download] 100% of   27.17MiB
downloaded {title}
